## Imports

In [20]:
import numpy as np
from numba import njit
from ROOT import TLorentzVector,TGenPhaseSpace,TFile,TTree,TVector3
import random

## Event Parameters

In [2]:
events=1 # Number of events
beamE=8.8   # Electron beam energy
days=50     # Days of beamtime
fprefix="test_py" # output file name prefix
targetType="p"

## Numerical Constants

In [3]:
# Mass of Target
# ---------------------------------------------
if(targetType=="p"):
    mD    = 0.938272       # mD is mProton
else:
    mD    =  1.875612928   # mD is mDeuteron
    
# Mass of Electron
# ---------------------------------------------
mE    = 0.000510999
m=mE

# Fine Structure Constant
# ---------------------------------------------
alpha_em = 1.0/137.036

# Luminosity
# ---------------------------------------------
lumi = 1.2e37 * 0.5 * 1.0e-24 * 1.0e-9 * 3600.0 * 24 * days

# Event Objects

In [4]:
# Phase Space Decay
# ---------------------------------------------
GenPhase = TGenPhaseSpace()

# 4-momentum of Particles
# ---------------------------------------------
eIn, dIn, eOut, dOut, q, dilepton, ePlus, eMinus = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()

# 4-momentum of Smeared Particles
# ---------------------------------------------
smear_dOut, smear_ePlus, smear_eMinus = TLorentzVector(), TLorentzVector(), TLorentzVector()

# Returned eventgen variables
# ---------------------------------------------
weight=np.array([0.0])
psf=np.array([0.0])
flux=np.array([0.0])
acc_ePlus=np.array([0.0])
acc_eMinus=np.array([0.0])
acc_hOut=np.array([0.0])
acc_photo=np.array([0.0])
gammaE_=np.array([0.0])
t_=np.array([0.0])
Mll2_=np.array([0.0])

# Macro for Branching the TTree

In [5]:
def branchTTree(outTree):
    outTree.Branch("ePlus",ePlus)
    outTree.Branch("eMinus",eMinus)
    outTree.Branch("hOut",dOut)
    outTree.Branch("smear_ePlus",smear_ePlus)
    outTree.Branch("smear_eMinus",smear_eMinus)
    outTree.Branch("smear_hOut",smear_dOut)
    outTree.Branch("q",q)
    outTree.Branch("weight",weight,"weight/D")
    outTree.Branch("psf",psf,"psf/D")
    outTree.Branch("flux",flux,"flux/D")
    outTree.Branch("acc_ePlus",acc_ePlus,"acc_ePlus/D")
    outTree.Branch("acc_eMinus",acc_eMinus,"acc_eMinus/D")
    outTree.Branch("acc_hOut",acc_hOut,"acc_hOut/D")
    outTree.Branch("acc_photo",acc_photo,"acc_photo/D")
    outTree.Branch("gammaE",gammaE_,"gammaE/D")
    outTree.Branch("t",t_,"t/D")
    outTree.Branch("Mll2",Mll2_,"Mll2/D")

## Functions

In [6]:
@njit
def Bremmstrahlung(kmin,kmax,beamE):
    ymin = kmin/beamE
    ymax = kmax/beamE
    y = random.uniform(ymin,ymax)
    flux = 0.01 * (4.0 / 3.0 * np.log(ymax / ymin) - 4.0 / 3.0 * (ymax - ymin) + 1.0 / 2.0 * (ymax * ymax - ymin * ymin))
    return flux, y*beamE

## Cross Section Model

In [7]:
@njit
def CE1(t,s,Mll2):
    return t*(s-mD**2)*(s-mD**2-Mll2+t)*(Mll2**2+6*Mll2*t+t**2+4*m**2*Mll2)+(Mll2-t)**2*(t**2*Mll2+mD**2*(Mll2+t)**2+4*m**2*mD**2*Mll2)
@njit
def CE2(t,s,Mll2):
    return -t*(s-mD**2)*(s-mD**2-Mll2+t)*(Mll2**2+t**2+4*m**2*(Mll2+2*t-2*m**2))+(Mll2-t)**2*(-mD**2*(Mll2**2+t**2)+2*m**2*(-t**2-2*mD**2*Mll2+4*m**2*mD**2))
@njit
def CM1(CE1,tauD,Mll2,t):
    return CE1-2*mD**2*(1+tauD)*(Mll2-t)**2*(Mll2**2+t**2+4*m**2*Mll2)
@njit
def CM2(CE2,tauD,Mll2,t):
    return CE2+2*mD**2*(1+tauD)*(Mll2-t)**2*(Mll2**2+t**2+4*m**2*(Mll2-t-2*m**2))

# Kinematics

In [8]:
@njit
def get_EC_ED_cth(EA,EB,MA,MB,MC,MD,t):
    # Assuming t > 0
    EC=(EA**2+2*EA*EB+EB**2+MC**2-MD**2)/(2*(EA+EB))
    ED=(EA**2+2*EA*EB+EB**2-MC**2+MD**2)/(2*(EA+EB))
    cth=(-t-MA**2-MD**2+2*EA*ED)/(2*np.sqrt(EA**2-MA**2)*np.sqrt(ED**2-MD**2))
    return EC,ED,cth
@njit
def get_tauD(t):
    return -t/(4*mD**2)
@njit
def get_s(Egamma):
    return mD**2+2*mD*Egamma
@njit
def get_cth_d_lab(Mll2,Egamma,t):
    tauD = get_tauD(t)
    s = get_s(Egamma)
    return (Mll2+2*(s+mD**2)*tauD)/(2*(s-mD**2)*np.sqrt(tauD*(1+tauD)))
@njit
def get_p_d_lab(t):
    return np.sqrt(((2*mD**2-t)/(2*mD))**2-mD**2)
@njit
def get_p_dilepton_lab(p_d_lab,cth_d_lab,Egamma):
    return (p_d_lab**2*np.sqrt(1-cth_d_lab**2)*np.abs(np.sqrt(1-cth_d_lab**2))+(Egamma-p_d_lab*cth_d_lab)**2)/np.sqrt(Egamma**2-2*Egamma*p_d_lab*cth_d_lab+p_d_lab**2)
@njit
def get_cth_dilepton_lab(p_d_lab,cth_d_lab,Egamma):
    return (Egamma-p_d_lab*cth_d_lab)/np.sqrt(Egamma**2-2*Egamma*p_d_lab*cth_d_lab+p_d_lab**2)
@njit
def get_beta(Mll2):
    return np.sqrt(1-4*m**2/Mll2)
@njit
def get_max_Mll2(Egamma,t):
    return 2*mD*Egamma+t-(Egamma/mD)*(2*mD**2-t-np.sqrt(t**2-4*mD**2*t))
@njit
def get_delta_T(tauD,t):
    # Approximation for small -t , large Mll2, large s
    return np.sqrt(-t*(1-tauD)-mD**2*tauD**2)
@njit
def get_delta_T2(s,Mll2,t,r):
    r0 = np.sqrt((s-mD**2)**2)
    cth_cm = (2 * s * (t - 2 * mD * mD) + (s + mD * mD)*(s + mD * mD - Mll2)) / (r0 * r)
    sth_cm = np.sqrt(1-cth_cm**2)
    return sth_cm*r/(2*np.sqrt(s))

# Form Factors

In [9]:
conv = 0.197327
a1,a2,a3,a4,als1,als2,als3,als4 = 1.57057 * conv**2, 12.23792 * conv**2, -42.04576 * conv**2, 27.92014 * conv**2, 1.52501 * conv**2, 8.75139 * conv**2,  15.97777 * conv**2, 23.20415* conv**2
b1,b2,b3,b4,bes1,bes2,bes3,bes4 = 0.07043 * conv, 0.14443 * conv, -0.27343 * conv, 0.05856 * conv, 43.67795 * conv**2, 30.05435 * conv**2, 16.43075 * conv**2,  2.80716* conv**2
c1,c2,c3,c4,gas1,gas2,gas3,gas4 = -0.16577, 0.27557 ,  -0.05382 , -0.05598 ,  1.87055 * conv**2, 14.95683 * conv**2,  28.04312 * conv**2, 41.12940* conv**2
@njit
def get_g0_g1_g2(Q2):
    g0 = a1/(als1+Q2)+a2/(als2+Q2)+a3/(als3+Q2)+a4/(als4+Q2)
    g1 = np.sqrt(Q2)*(b1/(bes1+Q2)+b2/(bes2+Q2)+b3/(bes3+Q2)+b4/(bes4+Q2))
    g2 = Q2 * (c1/(gas1+Q2)+c2/(gas2+Q2)+c3/(gas3+Q2)+c4/(gas4+Q2))
    return g0,g1,g2
@njit
def get_GC_GQ_GM(t):
    #t_fm = t/0.0389 # GeV^2 --> fm^-1
    tau = get_tauD(t)
    g0,g1,g2 = get_g0_g1_g2(-t)
    GC = 1/(1-t/4/0.807)**4/(2*tau+1)*((1-2/3*tau)*g0+8/3*np.sqrt(2*tau)*g1+2/3*(2*tau-1)*g2)
    GQ = 1/(1-t/4/0.807)**4/(2*tau+1)*(-g0+np.sqrt(2/tau)*g1-(tau+1)/tau*g2)
    GM = 1/(1-t/4/0.807)**4/(2*tau+1)*(2*g0+2*(2*tau-1)/np.sqrt(2*tau)*g1-2*g2)
    return GC,GQ,GM

# Cross Section

In [10]:
@njit
def get_dsigma_dt_dMll2(t,Egamma,Mll2):
    B = get_beta(Mll2)
    s = get_s(Egamma)
    tauD = get_tauD(t)
    ce1 = CE1(t,s,Mll2)
    ce2 = CE2(t,s,Mll2)
    cm1 = CM1(ce1,tauD,Mll2,t)
    cm2 = CM2(ce2,tauD,Mll2,t)
    CE = ce1 + (ce2/B)*np.log((1+B)/(1-B))
    CM = cm1 + (cm2/B)*np.log((1+B)/(1-B))
    GC,GQ,GM = get_GC_GQ_GM(t)
    factor1 = (4*alpha_em**3*B)/((s-mD**2)**2*t**2*(Mll2-t)**4)
    factor2 = (CE*(GC**2+8/9*tauD**2*GQ**2)+CM*2/3*tauD*GM**2)
    return factor1*factor2*(0.389379*10**6) # nb/GeV^4
@njit
def get_dsigma_dt_dMll2_dcosth(t,Egamma,Mll2,theta,phi):
    beta = get_beta(Mll2)
    s = get_s(Egamma)
    tauD = get_tauD(t)
    r = np.sqrt((s-mD**2-Mll2)**2-4*mD**2*Mll2)
    deltaT = get_delta_T2(s,Mll2,t,r)
    a = beta * r * np.cos(theta)
    b = beta * ((Mll2*(s-mD**2-Mll2)+t*(s-mD**2+Mll2))/r)*np.cos(theta)-beta*((2*(s-mD**2)*np.sqrt(Mll2)*deltaT)/r)*np.sin(theta)*np.cos(phi)
    L = ((Mll2-t)**2-b**2)/4
    A = (s-mD**2)**2*deltaT**2-t*a*(a+b)-mD**2*b**2-t*(4*mD**2-t)*Mll2+mE**2/L*(((Mll2-t)*(a+b)-(s-mD**2)*b)**2+t*(4*mD**2-t)*(Mll2-t)**2)
    B = (Mll2+t)**2 + b**2 + 8 * mE**2 * Mll2 - 4*mE**2*(t+2*mE**2)/L*(Mll2-t)**2
    F1p = (1. / ((1 - t / 0.71)*(1 - t / 0.71)))*(1 / (1 - t / (4. * mD * mD)))*(1 - 2.79 * t / (4 * mD * mD))
    F2p = (1. / ((1 - t / 0.71)*(1 - t / 0.71)))*(1 / (1 - t / (4. * mD * mD)))*(2.793 - 1)
    factor1 = (alpha_em**3*beta)/(4*np.pi*(s-mD**2)**2*(-t)*L)
    factor2 = (F1p**2-t/4/mD**2*F2p**2)*A/(-t)+(F1p+F2p)**2*B/2
    return factor1*factor2*(0.389379*10**6) # nb/GeV^4

## Acceptance

In [11]:
facc_solid = TFile("acceptance/acceptance_solid_JPsi_electron_target315_output.root","r")
acc_ele_solid = facc_solid.Get("acceptance_ThetaP_overall")
def acc_e(P):
    binx = acc_ele_solid.GetXaxis().FindBin(P.Theta()*180.0/np.pi)
    biny = acc_ele_solid.GetYaxis().FindBin(P.P())
    return acc_ele_solid.GetBinContent(binx,biny)

def acc_D(P):
    binx = acc_ele_solid.GetXaxis().FindBin(P.Theta()*180.0/np.pi)
    biny = acc_ele_solid.GetYaxis().FindBin(P.P())
    return acc_ele_solid.GetBinContent(binx,biny)

# Smearing

In [12]:
fres_electron_solid = TFile("acceptance/JPsi_electron_resolution_2d.root","READ")
fres_positron_solid = TFile("acceptance/JPsi_electron_resolution_2d.root","READ")
fres_proton_solid = TFile("acceptance/JPsi_proton_resolution_2d.root","READ")
res_electron_solid_p = fres_electron_solid.Get("p_resolution")
res_positron_solid_p = fres_positron_solid.Get("p_resolution")
res_proton_solid_p = fres_proton_solid.Get("p_resolution")
res_electron_solid_th = fres_electron_solid.Get("theta_resolution")
res_positron_solid_th = fres_positron_solid.Get("theta_resolution")
res_proton_solid_th = fres_proton_solid.Get("theta_resolution")
res_electron_solid_phi = fres_electron_solid.Get("phi_resolution")
res_positron_solid_phi = fres_positron_solid.Get("phi_resolution")
res_proton_solid_phi = fres_proton_solid.Get("phi_resolution")
res_electron_solid_p.Scale(1.5)
res_positron_solid_p.Scale(1.5)
res_proton_solid_p.Scale(1.5)
res_electron_solid_th.Scale(1.5)
res_positron_solid_th.Scale(1.5)
res_proton_solid_th.Scale(1.5)
res_electron_solid_phi.Scale(1.5)
res_positron_solid_phi.Scale(1.5)
res_proton_solid_phi.Scale(1.5)

In [13]:
def smear_particle(P,particleType):
    p = P.P()
    th = P.Theta()
    phi= P.Phi()
    if(particleType=="e+" or particleType=="e-"):
        dp = res_positron_solid_p.GetBinContent(res_positron_solid_p.FindBin(p,th*180.0/np.pi))/100.0
        dth = res_positron_solid_th.GetBinContent(res_positron_solid_th.FindBin(p,th*180.0/np.pi))/100.0
        dphi = res_positron_solid_phi.GetBinContent(res_positron_solid_phi.FindBin(p,th*180.0/np.pi))/100.0
    elif(particleType=="p" or particleType=="d"):
        dp = res_proton_solid_p.GetBinContent(res_proton_solid_p.FindBin(p,th*180.0/np.pi))/100.0
        dth = res_proton_solid_th.GetBinContent(res_proton_solid_th.FindBin(p,th*180.0/np.pi))/100.0
        dphi = res_proton_solid_phi.GetBinContent(res_proton_solid_phi.FindBin(p,th*180.0/np.pi))/100.0
    pprime = p*np.random.normal(1,dp)
    thprime = th*np.random.normal(1,dth)
    phiprime = phi*np.random.normal(1,dphi)
    smear_P = TLorentzVector(pprime*np.sin(thprime)*np.cos(phiprime),
                            pprime*np.sin(thprime)*np.sin(phiprime),
                            pprime*np.cos(thprime),
                            np.sqrt(pprime**2+P.M2()))
    return smear_P
smear_particle(TLorentzVector(1,2,3,4),"e+").Print()

(x,y,z,t)=(1.000000,2.000000,3.000000,4.000000) (P,eta,phi,E)=(3.741657,1.103587,1.107149,4.000000)


# Kinematic Limits

In [14]:
tmin,tmax=-1.5,-1.0
Mll2min,Mll2max=4,9
kmin=6   # Minimum photon energy
kmax=8   # Maximum photon energy

## Generator

In [15]:
%%time
# Generate file name
# ------------------------------
fname="./data/{}_e{}_beamE_{:.2f}_evts_{}.root".format(fprefix,targetType,beamE,events)
# Generate TFile
# ------------------------------
outFile=TFile(fname,"RECREATE")
# Generate TTree
# ------------------------------
outTree=TTree("tree","tree")
# Create the branches for the TTree
# ------------------------------
#outTree.Branch("ePlus",ePlus)
#outTree.Branch("eMinus",eMinus)
outTree.Branch("hOut",dOut)

outTree.Branch("smear_ePlus",smear_ePlus)
outTree.Branch("smear_eMinus",smear_eMinus)
outTree.Branch("smear_hOut",smear_dOut)

outTree.Branch("q",q)
outTree.Branch("weight",weight,"weight/D")
outTree.Branch("psf",psf,"psf/D")
outTree.Branch("flux",flux,"flux/D")
outTree.Branch("acc_ePlus",acc_ePlus,"acc_ePlus/D")
outTree.Branch("acc_eMinus",acc_eMinus,"acc_eMinus/D")
outTree.Branch("acc_hOut",acc_hOut,"acc_hOut/D")
outTree.Branch("acc_photo",acc_photo,"acc_photo/D")
outTree.Branch("gammaE",gammaE_,"gammaE/D")
outTree.Branch("t",t_,"t/D")

outTree.Branch("Mll2",Mll2_,"Mll2/D")
# Print event information
# ------------------------------

print("Beginning Monte Carlo Simulation of e({:.2f} GeV)+{} --> gamma + {} --> e- + e+ + {}'".format(beamE,targetType,targetType,targetType))
print("----------------------------")
print("Event Kinematics")
print(np.abs(tmax), "< |t| <",np.abs(tmin),"[GeV^2]")
print(Mll2min,"< Mll2 <",Mll2max,"[GeV^2]")
print(kmin,"< Egamma <",kmax,"[GeV]")
print("----------------------------")
print("Number of Events =",events)
print("Luminosity =", np.round(lumi/3600.0/24/days,2),"(events/nb/s)")
print("Integrated Luminosity =",np.round(lumi,2),"(events/nb)")

success=0

for evt in range(events):
    if( (evt+1)%(events/10)==0):
        print(evt+1,"out of",events)
    # Set Initial Vectors
    eIn.SetPxPyPzE(0,0,np.sqrt(beamE**2-mE**2),beamE)
    dIn.SetXYZM(0,0,0,mD)
    
    # Do Bremmstrahlung
    flux[0], gammaE = Bremmstrahlung(kmin,kmax,beamE)
    q.SetPxPyPzE(0,0,gammaE,gammaE)
    
    # Get scattered electron
    eOut = eIn-q
    
    # Generate t from allowable range
    t = random.uniform(tmin,tmax)
    
    # Generate Mll2 from allowable range
    Mll2max0 = get_max_Mll2(gammaE,t)
    if(Mll2max<Mll2max0):
        Mll2max0=Mll2max
    Mll2 = random.uniform(Mll2min,Mll2max0)
    
    # Set Lab Frame TLorentzVectors
    phi=random.uniform(0,2*np.pi)
    pDeuteron = get_p_d_lab(t)
    cthDeuteron = get_cth_d_lab(Mll2,gammaE,t)
    pDilepton = get_p_dilepton_lab(pDeuteron,cthDeuteron,gammaE)
    cthDilepton = get_cth_dilepton_lab(pDeuteron,cthDeuteron,gammaE)
    
    dOut.SetPxPyPzE(pDeuteron*np.sqrt(1-cthDeuteron**2)*np.cos(phi),
                    pDeuteron*np.sqrt(1-cthDeuteron**2)*np.sin(phi),
                    pDeuteron*cthDeuteron,
                    np.sqrt(pDeuteron**2+mD**2))
    
    dilepton.SetPxPyPzE(-pDilepton*np.sqrt(1-cthDilepton**2)*np.cos(phi),
                        -pDilepton*np.sqrt(1-cthDilepton**2)*np.sin(phi),
                        pDilepton*cthDilepton,
                        np.sqrt(pDilepton**2+Mll2))
    
    # Decay the Dilepton
    GenPhase.SetDecay(dilepton,2,np.array([m,m]))
    GenPhase.Generate()
    ePlus,eMinus = GenPhase.GetDecay(0), GenPhase.GetDecay(1)
    comBOOST=dilepton.BoostVector()
    
    ePlus.Boost(-comBOOST)
    dOut.Boost(-comBOOST)
    Zhat=-dOut.Vect()
    
    dilepton.Print()
    dOut.Print()
    # Get dsdt of (gamma + D --> e-e+ + D)
    
    weight[0] = get_dsigma_dt_dMll2_dcosth(t,gammaE,Mll2,theta,phi)
    
    # Get phase space factor
    psf[0] = (tmax-tmin)*(Mll2max0-Mll2min)*4*np.pi
    
    # Get acceptances of final state particles
    acc_ePlus[0] = acc_e(ePlus)
    acc_eMinus[0]= acc_e(eMinus)
    acc_hOut[0]  = acc_D(dOut)
    acc_photo[0]   = acc_ePlus[0]*acc_eMinus[0]
    
    # Store event variables
    gammaE_[0]   = gammaE
    t_[0]        = t
    Mll2_[0]     = Mll2
    
    # Smear final state particles if allowed based on acceptance
    if(acc_ePlus[0]>0):
        temp = smear_particle(ePlus,"e+")
        smear_ePlus.SetPx(temp.Px())
        smear_ePlus.SetPy(temp.Py())
        smear_ePlus.SetPz(temp.Pz())
        smear_ePlus.SetE(temp.E())
    if(acc_eMinus[0]>0):
        temp = smear_particle(eMinus,"e-")
        smear_eMinus.SetPx(temp.Px())
        smear_eMinus.SetPy(temp.Py())
        smear_eMinus.SetPz(temp.Pz())
        smear_eMinus.SetE(temp.E())
    if(acc_hOut[0]>0):
        temp = smear_particle(dOut,targetType)
        smear_dOut.SetPx(temp.Px())
        smear_dOut.SetPy(temp.Py())
        smear_dOut.SetPz(temp.Pz())
        smear_dOut.SetE(temp.E())
        
    # Fill the tree only when weight > 0
    if(weight[0]>0.0):
        success+=1
        outTree.Fill()
        
print("----------------------------")
print("Number of Sucesses =",success,"out of",events,"events")
print("Monte Carlo generated a valid event",np.round(success/events*100,2),"% of the time")
print("----------------------------")

Beginning Monte Carlo Simulation of e(8.80 GeV)+p --> gamma + p --> e- + e+ + p'
----------------------------
Event Kinematics
1.0 < |t| < 1.5 [GeV^2]
4 < Mll2 < 9 [GeV^2]
6 < Egamma < 8 [GeV]
----------------------------
Number of Events = 1
Luminosity = 6000.0 (events/nb/s)
Integrated Luminosity = 25920000000.0 (events/nb)


NameError: name 'theta' is not defined

(x,y,z,t)=(0.000000,0.000000,-0.000000,2.293262) (P,eta,phi,E)=(0.000000,-5.545193,0.000000,2.293262)
(x,y,z,t)=(0.649889,-1.003580,-1.940926,2.465173) (P,eta,phi,E)=(2.279632,-1.261295,-0.996130,2.465173)


In [21]:
dOut.Print()
dOut.Angle(TVector3(0,0,1))

2.58948455878268

(x,y,z,t)=(0.649889,-1.003580,-1.940926,2.465173) (P,eta,phi,E)=(2.279632,-1.261295,-0.996130,2.465173)
